In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
from sklearn.cluster import KMeans
import numpy as np
from PIL import Image
import pandas as pd

In [ ]:
# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Matplotlib의 폰트를 Nanum 폰트로 설정
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

# 폰트가 제대로 로드되었는지 확인
plt.figure(figsize=(10, 2))
plt.title('한글 테스트: 폰트가 정상적으로 작동합니다')
plt.show()


- Directory Path 저장

In [ ]:
# 코드를 공유드라이브로 옮길 경우, 아래 디렉토리 수정 필요
# input_folder = "/content/drive/MyDrive/비라제 E팀 공유 폴더/전처리 파일/"
# output_folder = "/content/drive/MyDrive/비라제 E팀 공유 폴더/색상 추출 파일/"

input_folder = "/content/drive/MyDrive/24-1/비주얼라이제이션/240506 test/전처리 파일/"
output_folder = "/content/drive/MyDrive/24-1/비주얼라이제이션/240506 test/색상 추출 파일/"

In [ ]:
# Directory path list
dir_list = ["21대 총선 더불어민주당", "21대 총선 미래통합당", "18대 총선 한나라당", "18대 총선 통합민주당",
            "8대 지선 기초 더불어민주당", "8대 지선 기초 국민의힘", "8대 지선 광역 더불어민주당", "8대 지선 광역 국민의힘",
            "7대 지선 기초 자유한국당", "7대 지선 기초 더불어민주당", "7대 지선 광역 자유한국당", "7대 지선 광역 더불어민주당",
            "4대 지선 기초 한나라당", "4대 지선 기초 열린우리당", "4대 지선 기초 민주당", "4대 지선 광역 한나라당", "4대 지선 광역 열린우리당", "4대 지선 광역 민주당",
            "3대 지선 기초 한나라당", "3대 지선 기초 새천년민주당", "3대 지선 광역 한나라당", "3대 지선 광역 새천년민주당"]

In [ ]:
# 다시....
dir_list_re = ["18대 총선 한나라당", "20대 총선 국민의당"]

# **1. 색상 추출하고 DataFrame에 저장**


In [ ]:
import re

# Step 1: Load Image Files
def load_images_from_directory(directory):
    images = []
    for file_name in os.listdir(directory):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(directory, file_name)
            try:
                image = Image.open(file_path).convert("RGB")
                images.append((file_name, image))
            except Exception as e:
                print(f"Error loading image {file_name}: {e}")
    return images

# Step 2: Apply K-Means Clustering
def extract_main_colors(image, num_colors, exclude_color_range=((0, 245, 0), (10, 255, 10))):
    # Convert image to numpy array
    image_np = np.array(image)

    # Exclude the specified color range (green mask)
    min_exclude, max_exclude = exclude_color_range
    mask = np.all(np.logical_and(image_np >= min_exclude, image_np <= max_exclude), axis=-1)
    pixels = image_np[~mask].reshape((-1, 3))

    # If there are no pixels left after excluding the specified color, return an empty list
    if len(pixels) == 0:
        return []

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=num_colors, init='k-means++')
    kmeans.fit(pixels)

    # Get the main colors and their labels
    main_colors = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_

    # Count the number of pixels for each label
    label_counts = np.bincount(labels)

    # Calculate the percentage of each main color
    total_pixels = len(labels)
    percentages = label_counts / total_pixels

    # Pair color codes with percentages
    main_colors_with_percentages = [(tuple(color), percentage) for color, percentage in zip(main_colors, percentages)]

    # Sort by percentage in descending order
    main_colors_with_percentages.sort(key=lambda x: x[1], reverse=True)

    return main_colors_with_percentages

# Step 3: Save results
def save_colors(image_name, main_colors):
    # Ensure that we have exactly 3 colors
    if len(main_colors) < 3:
        main_colors += [((0, 0, 0), 0.0)] * (3 - len(main_colors))  # Fill with black if fewer than 3 colors

    # Flatten the color and percentage pairs
    flattened_data = [image_name]
    for color, percentage in main_colors:
        flattened_data.extend([color, round(percentage, 4)])  # Round percentage to 4 decimal places

    return flattened_data

# Step 4: Parse image filename into separate columns
def parse_image_filename(filename):
    # Remove file extension
    base_name = os.path.splitext(filename)[0]
    # Split by underscore and strip any leading/trailing spaces
    parts = [part.strip() for part in base_name.split('_')]
    # Ensure that the parsed information fits into our expected columns
    if len(parts) != 9:
        print(f"Warning: Filename {filename} does not match expected format.")
        parts += [''] * (9 - len(parts))  # Fill with empty strings if fewer than 9 parts
    return parts

In [ ]:
# 색상 추출 및 저장 단계
# Load images from directory
for name in dir_list_re :
    input_dir = input_folder + name
    output_dir = output_folder + name

    images = load_images_from_directory(input_dir)

    # 색상 추출
    num_main_colors = 3
    df = []
    for file_name, image in images:
        main_colors = extract_main_colors(image, num_main_colors)
        main_colors_per_image = save_colors(file_name, main_colors)
        df.append(main_colors_per_image)

    # Create DataFrame with color information
    columns = ['Image File Name', 'Color Code 1', 'Percentage 1', 'Color Code 2', 'Percentage 2', 'Color Code 3', 'Percentage 3']
    df = pd.DataFrame(df, columns=columns)

    # Parse image filename into separate columns and add them to the DataFrame
    parsed_columns = ['연도', '대', '의원', '지역', '선거구', '기호', '당', '당선여부', '후보자 이름']
    parsed_data = df['Image File Name'].apply(parse_image_filename)
    parsed_df = pd.DataFrame(parsed_data.tolist(), columns=parsed_columns)

    # Concatenate the parsed columns with the original DataFrame
    final_df = pd.concat([parsed_df, df], axis=1).drop(columns=['Image File Name'])

    # final Dataframe 저장
    output_path = output_dir + "_color_extraction.csv"
    final_df.to_csv(output_path, index=False)
    print(f"Parsed image information and extracted colors saved successfully to {output_path}.")

In [ ]:
final_df.head()

# **2. 색상 추출 DataFrame과 득표율 DataFrame 합치기**

In [ ]:
# 코드를 공유드라이브로 옮길 경우, 아래 디렉토리 수정 필요
# vote_folder = "/content/drive/MyDrive/비라제 E팀 공유 폴더/선거결과 편집 데이터/"
vote_folder = "/content/drive/MyDrive/24-1/비주얼라이제이션/240506 test/선거결과 편집 데이터/"

# 한글 인코딩 문제 해결
import unicodedata

In [ ]:
# 색상 추출 데이터 프레임에 문제 없는지 확인..
# 아래 코드 돌리다보니 문제가 있어서......
for name in dir_list:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # '당' 컬럼의 유일한 값 추출
    unique_parties = final_df['당'].unique()

    # '당' 컬럼의 값이 1개 초과인지 확인
    if len(unique_parties) > 1:
        print(f"Directory '{name}' has a final_df with more than one party value: {unique_parties}")
    else:
        print(f"Directory '{name}' has a final_df with exactly one party value: {unique_parties}")

In [ ]:
# 20대 총선 국민의당, 18대 총선 한나라당에 당명 다른 것이 섞여 있는 것 해결
# 데이터 파일 경로 설정
directories = {
    '20대 총선 국민의당': '국민의당',
    '18대 총선 한나라당': '한나라당'
}

for directory_name, party_name in directories.items():
    directory = output_folder + directory_name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # '당' 컬럼 정규화
    final_df['당'] = final_df['당'].apply(lambda x: unicodedata.normalize('NFC', x))

    # 특정 party 값으로 필터링
    filtered_df = final_df[final_df['당'] == party_name]

    # 필터링 결과 확인
    if not filtered_df.empty:
        print(f"Filtered data for {directory_name} contains {len(filtered_df)} records with party: {party_name}")
        filtered_df.to_csv(final_df_path, index=False, encoding='utf-8')
        print(f"Filtered data for {directory_name} saved successfully with party: {party_name}")
    else:
        print(f"No data remaining after filtering for {directory_name} with party: {party_name}. Check the party names and data.")



- Missing data 저장할 directory 생성

In [ ]:
# missing data 저장하자..
# 생성하고자 하는 폴더 이름
new_folder_name = "missing data"

# 최종 경로 생성
full_path = os.path.join(output_folder, new_folder_name)

# 디렉토리 생성
os.makedirs(full_path, exist_ok=True)

print(f"Folder '{new_folder_name}' created successfully in '{full_path}'.")

## 2-1. 총선

### 2-1-1. 21대 총선 (2020년)

In [ ]:
import pandas as pd
import unicodedata

# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2020_legislative.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2020 = ["21대 총선 더불어민주당", "21대 총선 미래통합당"]
for name in dir_2020:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print(f"Missing data entries from vote data not present in color data for {name}:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")


### 2-1-2. 20대 총선 (2016년)

- 국민의 당 득표 데이터 없음

In [ ]:
# 20대 총선
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2016_legislative.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2016 = ["20대 총선 새누리당", "20대 총선 더불어민주당", "20대 총선 국민의당"]
for name in dir_2016:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-1-3. 19대 총선 (2012년)

In [ ]:
# 19대 총선
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2012_legislative.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2012 = ["19대 총선 새누리당", "19대 총선 민주통합당"]
for name in dir_2012:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-1-4. 18대 총선 (2008년)


In [ ]:
# 18대 총선
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2008_legislative.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2008 = ["18대 총선 한나라당", "18대 총선 통합민주당"]
for name in dir_2008:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-1-5. 17대 총선 (2004년)

In [ ]:
# 17대 총선
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2004_legislative.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2004 = ["17대 총선 한나라당", "17대 총선 열린우리당"]
for name in dir_2004:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-1-6. 16대 총선 (2000년)

In [ ]:
# 16대 총선
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2000_legislative.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2000 = ["16대 총선 한나라당", "16대 총선 새천년민주당"]
for name in dir_2000:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

## 2-2. 지선

### 2-2-1. 8대 지선 (2022년)

In [ ]:
# 8대 지선 기초
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2022_local.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2022 = ["8대 지선 기초 더불어민주당", "8대 지선 기초 국민의힘"]
for name in dir_2022:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

In [ ]:
# 8대 지선 광역
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2022_local_met.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2022_met = ["8대 지선 광역 더불어민주당", "8대 지선 광역 국민의힘"]
for name in dir_2022_met:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 지선 광역은 포스터 이름과 선거데이터의 선거구 이름이 동일하지 않음. inner join에서 선거구 빼야 함
    merged_df_normalized = pd.merge(final_df, vote_df, on=['당','후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-2-2. 7대 지선 (2018년)

In [ ]:
# 7대 지선 기초
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2018_local.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2018 = ["7대 지선 기초 자유한국당", "7대 지선 기초 더불어민주당"]
for name in dir_2018:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

In [ ]:
# 7대 지선 광역
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2018_local_met.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2018_met = ["7대 지선 광역 자유한국당", "7대 지선 광역 더불어민주당"]
for name in dir_2018_met:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 지선 광역은 포스터 이름과 선거데이터의 선거구 이름이 동일하지 않음. inner join에서 선거구 빼야 함
    merged_df_normalized = pd.merge(final_df, vote_df, on=['당','후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-2-3. 6대 지선 (2014년)

In [ ]:
# 6대 지선 기초
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2014_local.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2014 = ["6대 지선 기초 새정치민주연합", "6대 지선 기초 새누리당"]
for name in dir_2014:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

In [ ]:
# 6대 지선 광역
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2014_local_met.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2014_met = ["6대 지선 광역 새정치민주연합", "6대 지선 광역 새누리당"]
for name in dir_2014_met:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 지선 광역은 포스터 이름과 선거데이터의 선거구 이름이 동일하지 않음. inner join에서 선거구 빼야 함
    merged_df_normalized = pd.merge(final_df, vote_df, on=['당','후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-2-4. 5대 지선 (2010년)

In [ ]:
# 5대 지선 기초
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2010_local.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2010 = ["5대 지선 기초 한나라당", "5대 지선 기초 민주당"]
for name in dir_2010:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

In [ ]:
# 5대 지선 광역
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2010_local_met.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2010_met = ["5대 지선 광역 한나라당", "5대 지선 광역 민주당"]
for name in dir_2010_met:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 지선 광역은 포스터 이름과 선거데이터의 선거구 이름이 동일하지 않음. inner join에서 선거구 빼야 함
    merged_df_normalized = pd.merge(final_df, vote_df, on=['당','후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-2-5. 4대 지선 (2006년)


In [ ]:
# 4대 지선 기초
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2006_local.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2006 = ["4대 지선 기초 한나라당", "4대 지선 기초 열린우리당", "4대 지선 기초 민주당"]
for name in dir_2006:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['선거구', '당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['선거구', '당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['선거구', '당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

In [ ]:
# 4대 지선 광역
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2006_local_met.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2006_met = ["4대 지선 광역 한나라당", "4대 지선 광역 열린우리당", "4대 지선 광역 민주당"]
for name in dir_2006_met:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 지선 광역은 포스터 이름과 선거데이터의 선거구 이름이 동일하지 않음. inner join에서 선거구 빼야 함
    merged_df_normalized = pd.merge(final_df, vote_df, on=['당','후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

### 2-2-6. 3대 지선 (2002년)

In [ ]:
# 3대 지선 기초
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2002_local.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2002 = ["3대 지선 기초 한나라당", "3대 지선 기초 새천년민주당"]
for name in dir_2002:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 데이터 병합
    merged_df_normalized = pd.merge(final_df, filtered_vote_df, on=['당', '후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")

In [ ]:
# 3대 지선 광역
# 선거 데이터 열 이름 바꾸기
vote_df_path = vote_folder + "final_2002_local_met.xlsx"
vote_df = pd.read_excel(vote_df_path, engine='openpyxl')
vote_df.rename(columns={'sggName': '선거구', 'party': '당', 'candidate': '후보자 이름', 'vote_percent': '득표율', 'margin_percent': '득표율 차'}, inplace=True)

# '선거구'에서 중앙점 제거
vote_df['선거구'] = vote_df['선거구'].str.replace('·', '')  # 중앙점 제거

# 모든 한국어 데이터 정규화
def normalize_text(df):
    text_columns = df.select_dtypes(include=[object]).columns  # object 타입의 모든 컬럼을 선택
    for col in text_columns:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFKC', str(x)) if pd.notnull(x) else x)
    return df

dir_2002_met = ["3대 지선 광역 한나라당", "3대 지선 광역 새천년민주당"]
for name in dir_2002_met:
    directory = output_folder + name
    final_df_path = directory + "_color_extraction.csv"
    final_df = pd.read_csv(final_df_path, encoding='utf-8')

    # 정규화
    final_df = normalize_text(final_df)
    vote_df = normalize_text(vote_df)

    # '당'으로 필터링
    party_filter = final_df['당'].unique()
    filtered_vote_df = vote_df[vote_df['당'].isin(party_filter)]

    # 데이터 행 수 비교 전 출력
    print(f"Filtered vote data rows for '{name}': {len(filtered_vote_df)}")
    print(f"'{name}' color data rows: {len(final_df)}")

    # 병합 전 '당', '선거구', '후보자 이름' 기준으로 누락 데이터 확인
    check_missing = pd.merge(filtered_vote_df, final_df, on=['당', '후보자 이름'], how='outer', indicator=True)
    missing_data = check_missing[check_missing['_merge'] == 'left_only']
    if not missing_data.empty:
        missing_count = len(missing_data)
        print("Missing data count: ", missing_count)
        print("Missing data entries from vote data not present in color data:")
        print(missing_data[['당', '후보자 이름']])
        # 누락된 데이터를 CSV 파일로 저장
        missing_data_directory = output_folder + "missing data/"
        missing_data_path = missing_data_directory + name + "_missing_data.csv"
        missing_data.to_csv(missing_data_path, index=False)
        print(f"Missing data saved to {missing_data_path}.")

    # 지선 광역은 포스터 이름과 선거데이터의 선거구 이름이 동일하지 않음. inner join에서 선거구 빼야 함
    merged_df_normalized = pd.merge(final_df, vote_df, on=['당','후보자 이름'], how='inner')

    # 'sgId' 열 제거
    merged_df = merged_df_normalized.drop('sgId', axis=1)

    # 병합된 데이터프레임 저장
    merged_output_path_normalized = directory + "_color_with_vote_data.csv"
    merged_df.to_csv(merged_output_path_normalized, index=False)
    print(f"Normalized merged data saved successfully to {merged_output_path_normalized}.")